In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 250 observations

In [2]:
expo = 250
# Setting the true model
f1, f2 = bs.loss_model("Gamma", ["r1", "m1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])
f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05], n_sim = 10))

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [3]:
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

array([-10.79364386, -10.70701413, -10.01397631])

In [4]:
# Model for the bulk distribution
body_model_names = ["Exp", "Gamma", "Weibull", "Lognormal", "Inverse-Weibull", "Inverse-Gamma", "Inverse-Gaussian", "Lomax", "Log-Logistic", "Burr"]
body_model_param_names = [ ["λ1"], ["r1", "m1"], ["k1", "β1"],
                          ["μ1", "σ1"], ["k1", "β1"], ["r1", "m1"], ["μ1", "λ1"], ["α1", "σ1"], ["β1", "σ1"], ["α1", "β1", "σ1"] ]

# Prior distributions over the parameters of the bulk distribution
body_model_priors= [ 
    [bs.prior_model('gamma',body_model_param_names[0][0], 1, 1)], 
     [bs.prior_model('gamma',body_model_param_names[1][0], 1, 1), bs.prior_model('gamma',body_model_param_names[1][1], 1, 1)],
    [bs.prior_model('gamma',body_model_param_names[2][0], 1, 1), bs.prior_model('gamma',body_model_param_names[2][1], 1, 1)],
    [bs.prior_model('normal',body_model_param_names[3][0], 0, 0.5), bs.prior_model('gamma',body_model_param_names[3][1], 1, 1)],
     [bs.prior_model('gamma',body_model_param_names[4][0], 1, 1), bs.prior_model('gamma',body_model_param_names[4][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[5][0], 1, 1), bs.prior_model('gamma',body_model_param_names[5][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[6][0], 1, 1), bs.prior_model('gamma',body_model_param_names[6][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[7][0], 1, 1), bs.prior_model('gamma',body_model_param_names[7][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[8][0], 1, 1), bs.prior_model('gamma',body_model_param_names[8][1], 1, 1)],
    [bs.prior_model('gamma',body_model_param_names[9][0], 1, 1), bs.prior_model('gamma',body_model_param_names[9][1], 1, 1), 
     bs.prior_model('gamma',body_model_param_names[9][2], 1, 1)]
]

# Model for the tail of the distribution
tail_model_names = ["Weibull", "Lognormal", "Log-Logistic", "Lomax", "Burr", "Pareto-Tail", "GPD-Tail", "Inverse-Gamma", "Inverse-Weibull", "Exp", "Gamma"]

tail_model_param_names = [["k2", "β2"], ["μ2", "σ2"], ["β2", "σ2"], ["α2", "σ2"], ["α2", "β2", "σ2"], ["α2"], ["ξ2","σ2"], ["r2", "m2"], ["k2", "β2"], ["λ2"], ["r2", "m2"]]

# Prior distributions over the parameters of the bulk distribution
tail_model_priors= [
                [bs.prior_model('gamma',tail_model_param_names[0][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[0][1], 1, 1)],
                [bs.prior_model('normal',tail_model_param_names[1][0], 0, 0.5), bs.prior_model('gamma',tail_model_param_names[1][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[2][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[2][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[3][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[3][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[4][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[4][1], 1, 1), bs.prior_model('gamma',tail_model_param_names[4][2], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[5][0], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[6][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[6][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[7][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[7][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[8][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[8][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[9][0], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[10][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[10][1], 1, 1)]
]

γ_prior = bs.prior_model('gamma',"γ", 1, 1)

#Splicing model type
splicing_types = ["continuous"]

# Setting the models
fs, f_names, prior_spliced_model = [], [], []
for i in range(len(body_model_names)):
    for j in range(len(tail_model_names)):
        for splicing_type in splicing_types:
            f1, f2 =  bs.loss_model(body_model_names[i], body_model_param_names[i]), bs.loss_model(tail_model_names[j], tail_model_param_names[j])
            fs.append(bs.spliced_loss_model(f1 , f2, splicing_type))
            f_names.append(body_model_names[i] +"_"+ tail_model_names[j]+"_"+splicing_type)
            if splicing_type == "disjoint": 
                prior_spliced_model.append(bs.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior, p_prior]))
            else:
                prior_spliced_model.append(bs.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior]))  
for f in fs:
    f.set_ppf(), f.set_cdf(), f.set_pdf() 
f_spliced_dic = dict(zip(f_names, fs))
prior_dic = dict(zip(f_names, prior_spliced_model))
len(fs)

110

In [6]:
nobs, n_sim = expo, 2
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
popSize, ρ, c, n_step_max, err, paralell, n_proc, verbose = 10, 1/2, 0.99, 25, 1e-6, False, 4, False
dfs = []
for k in range(n_sim):
    print("Simulation #"+str(k))
    def fit_spliced_models(i):
        trace, log_marg, DIC, WAIC = bs.smc(Xs[k], fs[i], popSize, prior_spliced_model[i], ρ, c,n_step_max, err, paralell, 4, verbose)
        VaRs = [fs[i].ppf(trace.mean().values, prob) for prob in [0.95, 0.99, 0.995]]
#         premiums = fs[i].PP(trace.mean().values), fs[i].XOLP(trace.mean().values, P, L)
#         PnLs = np.array(fs[i].PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05], n_sim = int(1e5)))
#         caps = np.quantile(PnLs, [0.005, 0.01, 0.05])
        Wass_dist = bs.compute_Wasserstein(Xs[k], fs[i], trace.mean().values, 1)
        return(np.array([k, f_names[i], nobs, trace["γ"].mean(), log_marg, Wass_dist] + VaRs))
    %time res = Parallel(n_jobs= 40)(delayed(fit_spliced_models)(i) for i in range(len(fs)))
    df = pd.DataFrame(res, columns = ["sim", "model_name", "nobs", "γ_map", "log_marg", "Wass_dist", "q95", "q99", "q995"])
    df[df.columns[2:]] = df[df.columns[2:]].astype(float)

    df["posterior_probability"] = np.exp(df["log_marg"] - np.max(df["log_marg"])) / np.sum(np.exp(df["log_marg"] - np.max(df["log_marg"]))) 
    dfs.append(df)


In [6]:
pd.concat(dfs).to_csv("../../Data/Simulations/simu_well_spec_"+str(expo)+".csv", sep=',')
with open('../../Data/Simulations/sim_data_'+str(expo)+'.obj', 'wb') as fp:
    pickle.dump(Xs, fp)

In [7]:
with open('../../Data/Simulations/sim_data_'+str(expo)+'.obj', 'wb') as fp:
    pickle.dump(Xs, fp)

In [11]:
favorite_color = pickle.load( open( "../../Data/Simulations/sim_data_250.obj", "rb" ) )
# for k in range(n_sim):
#     best_models= df_final[df_final.sim == str(k)].sort_values(by='log_marg', ascending=False).iloc[:10]
#     print(best_models)

In [12]:
favorite_color

[array([1.52164909e-01, 7.32371844e-01, 1.19857396e-01, 8.25188794e-01,
        1.47358469e+00, 3.02820324e+00, 2.12038408e+01, 1.75394423e-01,
        1.00086669e+01, 1.19620566e+00, 4.96739655e-02, 3.48922355e+00,
        1.61145149e-01, 2.25610034e+00, 1.45157522e+00, 1.11443149e-01,
        2.26185748e-03, 1.24909844e+00, 1.21273885e-01, 1.91641645e+00,
        3.69097571e-01, 5.96121010e-01, 5.69808271e-02, 3.83735087e-03,
        1.74101491e-01, 1.05766239e+00, 3.23558300e-01, 1.31891744e+00,
        6.07664023e-02, 2.90487099e-03, 6.08133916e-02, 3.30596157e-01,
        3.33273758e+00, 1.09723322e+00, 8.95495436e-01, 1.08143316e-01,
        1.40878058e+00, 2.99406093e-01, 2.09052785e-01, 8.18320313e-01,
        6.08945240e-01, 1.49347947e+00, 1.00643786e+00, 6.50493899e-01,
        7.09283667e-01, 8.84273743e-03, 4.98292382e-01, 1.03501848e-02,
        5.51539029e-01, 6.76489812e-01, 2.46849177e+00, 5.32326884e-02,
        4.84925076e-01, 1.64120530e+00, 9.80026582e+00, 1.579595